In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data.head()

# **Exploratory Data Analysis**

In [ ]:
#Lets do some visualizations to see how have things worked out
data['ssc_b'].value_counts(normalize=True).plot(kind='bar')

In [ ]:
data['hsc_b'].value_counts(normalize=True).plot(kind='bar')

In [ ]:
data['workex'].value_counts(normalize=True)

In [ ]:
import seaborn as sns
pd.crosstab(data["status"],data[("specialisation")]).apply(lambda r: (r/r.sum())*100, axis=0)

# Mkt&HR is not a field where its easy to get a job

In [ ]:
#see the relationship of predictor with the outcome variable

sns.countplot(x="specialisation",hue="workex",data=data)

##It is visible that candidates with workex in Mkt&Fin had an advantage

# **Feature Engineering**

In [ ]:
data.head()

In [ ]:
data=data.drop(['sl_no'],axis=1)

In [ ]:
data.shape

In [ ]:
numerical_cols=pd.DataFrame(data.describe()).columns
numerical_cols
output_col='salary' #toggle between salary and status

In [ ]:
cat_data=data.drop(numerical_cols,axis=1)
cat_data.head()

In [ ]:
num_data=data[numerical_cols]
num_data.head()

In [ ]:
#dealing with num data first
num_data['salary']=num_data['salary'].fillna(0)
#print(num_data.isna().sum())
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(num_data)
scaled_num_data=pd.DataFrame(scaler.transform(num_data),columns=numerical_cols)
    

In [ ]:

#keeping an eye for data leakage
#scaled_num_data_wo_sal=scaled_num_data.drop('salary',axis=1)
cat_data=cat_data.drop('status',axis=1)

In [ ]:
#dealing with categorical data
cat_data.head()

In [ ]:
cat_data_dummy=pd.get_dummies(cat_data,drop_first=True)

In [ ]:
cat_data_dummy.shape

In [ ]:
cat_data_dummy.head()

In [ ]:
final_data=pd.concat([cat_data_dummy,scaled_num_data],axis=1)
#scaled_num_data_wo_sal

In [ ]:
output=final_data[['salary']]
train_final=final_data.drop(['salary'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_final, output, test_size=0.30, random_state=1)

In [ ]:
y_test.head()

# **Model Fitting**

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#model=RandomForestClassifier(n_estimators=100, random_state=0)
#model.fit(X_train,y_train)

from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train,y_train)

In [ ]:
preds=model.predict(X_test)
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score
#confusion_matrix(y_test, preds)
#print("The Accuracy score is: ",accuracy_score(y_test,preds)*100)
from sklearn.metrics import r2_score
r2_score(y_test, preds)

In [ ]:
importance=model.feature_importances_
names=X_train.columns

In [ ]:
feature_importance = np.array(importance)
feature_names = np.array(names)
data={'feature_names':feature_names,'feature_importance':feature_importance}
fi_df = pd.DataFrame(data)

#Sort the DataFrame in order decreasing feature importance
fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

In [ ]:
fi_df

In [ ]:
#fucking shit, my life was a lie!

# **Inspecting for Salary**

In [ ]:
#Uncomment for categorical problem
# import shap
# explainer = shap.TreeExplainer(model)
# X_train.loc[[10]]
# choosen_instance = X_train.loc[[10]]
# shap_values = explainer.shap_values(choosen_instance)
# shap.initjs()
# shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)
# shap.initjs()
# shap.summary_plot(shap_values, features=X_train, feature_names=X_train.columns)

In [ ]:
from time import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor
tic = time()
est = make_pipeline(QuantileTransformer(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 learning_rate_init=0.01,
                                 early_stopping=True))
est.fit(X_train, y_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.2f}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence


In [ ]:
numerical_cols_wo_salary=[x for x in numerical_cols if x!='salary']
numerical_cols_wo_salary

In [ ]:

print('Computing partial dependence plots...')
tic = time()
features = numerical_cols_wo_salary
display = plot_partial_dependence(
       est, X_train, features, kind="both", subsample=50,
       n_jobs=3, grid_resolution=20, random_state=0
)
print(f"done in {time() - tic:.3f}s")

display.figure_.subplots_adjust(hspace=0.3)

In [ ]:

print('Computing partial dependence plots...')
tic = time()
features = X_train.columns
display = plot_partial_dependence(
       est, X_train, features, kind="both", subsample=50,
       n_jobs=3, grid_resolution=20, random_state=0
)
print(f"done in {time() - tic:.3f}s")

display.figure_.subplots_adjust(hspace=0.3)